# Covariant encoding via RF states, based on 5-qubit code
- RF states in this code is 'N' copies of Bell state
- The performance is measured by 'entanglement error'
    1. Note: 'entanglement error = 1-Fent' for qubit covariant channel 

### Note: In this program, we calculate entanglement fidelity
- For qubit covariant channel:
    1. entanglement error = 1-Fent'
    2. worst case error <= 2 * entanglement error

## Outline
- Function part
    1. Used modules and some matrices
    2. Generate random SU(2) matrix
    3. From $|U_g / \sqrt{2} >> ^{\otimes N} $, get final measurement outcome $U_h$
    3. ' $|U_g / \sqrt{2} >> ^{\otimes N} $ pass through IID p-depolarizing, get final measurement outcome $U_h$
    4. About coding -- functions related to 5-qubit code
    5. About coding -- get the state after Ug encoding关于code部分 -- 得到经过encoding state
    6. About coding -- after noise and Uh decoding, calculate the entanglement error
- Numerical experiment part
    1. erasure noise
    2. completely depolarizing noise (one fifth RF states pass through noise)
    3. completely dephasing noise (one fifth RF states pass through noise)
    4. IID p-depolarizing noise (every RF state pass through p-depolarizing noise)

In [ ]:
# 5-qubit code
# Input: 1 logical qubit，1 ancilla qubit，Output: 4 measure qubits, 5 code qubits, 1 ancilla qubit

## Function Part

### 1. Used modules and some matrices

In [ ]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt  # For plot
from scipy.optimize import curve_fit  # For fitting
from scipy.stats import unitary_group  # Generate random unitary matrix


# get measure outcome via optimization, with the help of "paddle quantum"
import paddle
from paddle_quantum.circuit import UAnsatz  # Create Quantum Circuit
from paddle_quantum.state import vec, vec_random  # Generate Quantum state, (vector)
from paddle_quantum.state import density_op, density_op_random, completely_mixed_computational  # Generate Quantum state, (matrix)
from paddle_quantum.utils import dagger  # Take dagger for paddle.tensor
from paddle import matmul, trace  # Calculate inner product and trace for paddle.tensor


In [ ]:
# Some matrices

# Bell state -- np ket
bell_state = np.array([[1,0,0,1]]) / (2**0.5)
bell_state = bell_state.conj().T

# Pauli matrices
pauli_x = np.array([[0.0, 1.0], [1.0, 0.0]])
pauli_y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
pauli_z = np.array([[1.0, 0.0], [0.0, -1.0]])

### 2. Generate random SU(2) matrix

In [ ]:
# 2. Generate random SU(2) martrix: -- U2_random_Euler_mod()

def Rz(alpha):
    # output U：np.array -- rotation around Z axis with angle alpha
    U = np.array([[0.+0.j, 0.+0.j],[0.+0.j, 0.+0.j]])
    U[0][0] = np.e ** (-1j * alpha/2)
    U[1][1] = np.e ** (1j * alpha/2)
    
    return U

def Ry(alpha):
    # output U：np.array -- rotation around Y axis with angle alpha
    U = np.array([[0.+0.j, 0.+0.j],[0.+0.j, 0.+0.j]])
    U[0][0] = np.cos(alpha/2)
    U[0][1] = -np.sin(alpha/2)
    U[1][0] = np.sin(alpha/2)
    U[1][1] = np.cos(alpha/2)
    
    return U

def U2_Euler(alpha, beta, gamma):
    # output U：np.array -- rotation with Euler angle representation
    # U(alpha, beta, gamma) = Z(gamma) @ Y(beta) @ Z(alpha)
    # Note: alpha, gamma \in [0,2pi], beta \in [0,pi]
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    return U



def random_sinx(a, b):
    # Gerenrate distribution sin(x)dx, x \in [a,b]
        # Note：sin(x) should be > 0 for x \in [a,b]
    # Generate random (x,y), x \in [a,b], y \in [0, maxy]
        # for (xi, yi), if 0<yi<f(xi), output xi
    ymax = 1  # sin(x) always smaller than 1
    while 1:
        x = random.uniform(a, b)
        y = random.uniform(0, 1)
        if y < np.sin(x):
            break
    return x,y

def U2_random_Euler_mod():
    # output U：np.array -- rotation with Euler angle representation
    # beta in distribution sin(beta)d(beta), alpha, gamma are uniform
    alpha = random.uniform(0, 2*np.pi)
    beta, anc = random_sinx(0, np.pi)
    gamma = random.uniform(0, 2*np.pi)
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    
    return U

### 3. From $|U_g / \sqrt{2} >> ^{\otimes N} $, get final measurement outcome $U_h$
- This part deals with RF states pass through erasure noise, or completely depolarizing noise, orcompletely dephasing noise
- For completely depolarizing noise, a majority vote algorithm will be first used to get rid of n noisy outcomes (here we suppose that the number of how many RF states pass through noise is known)
- The final Uh from N measure outcomes {Uh1, ... UhN} is derived via paddle quantum, by optimizing loss function: L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh|Uhi>> |^2

In [ ]:
# 3. From Ug get final Uh
# 3.1 Generate a list of Uhi from distribution p(h|g) = 1/4 * | <<Ug|Uh>> |^2
# 3.2 For depolarizing noise, get rid of n noisy outcomes
# 3.3 For dephasing noise, get rid of n noisy outcomes

# 3.1 Generate a list of Uhi from distribution p(h|g) = 1/4 * | <<Ug|Uh>> |^2
def random_fromg_geth(Ug):
    # Input: Ug: np.matrix, distribution
    # Output: Uh:np.matrix: distribution -- p(h|g) = 1/4 * | <<Ug|Uh>> |^2
    pgh_max = 1  # p(h|g) always smaller than 1
    while 1:
        Uh = U2_random_Euler_mod()
        pgh = random.uniform(0, pgh_max)
        fgh = 1/4 * ((Ug.conj().T @ Uh).trace()) * ((Ug.conj().T @ Uh).trace().conj())  # 1/4 * | <<Ug|Uh>> |^2
        fgh = np.real(fgh)
        if pgh < fgh:
            break
    return Uh

def measoutcome_gN(Ug, N):
    # Input: Ug: np.matrix, distribution
    # Output: a list {Uh1, ... UhN}:np.matrix: distribution -- p(h|g) = 1/4 * | <<Ug|Uh>> |^2
    Uh_list = []
    for i in range(N):
        Uh = random_fromg_geth(Ug)
        Uh_list.append(Uh)
    
    return Uh_list

# 3.2 For depolarizing noise, get rid of n noisy outcomes
def getrid_dep(Uh_list, N, n):
    # Input: Uh_list: A list contain N+n elements: np.matrix
        # N follows distribution p(h|g) = 1/4 * | <<Ug|Uh>> |^2, n are randomly created as comes from maximally mixed state
    # Output: Uh_list_final -- get rid of n noisy outcomes: np.matrix
        # Method: For every Uhi, calculate Sum_j 1/4 * <<Uhi|Uhj>><<Uhj|Uhi>>, eliminate the smallest n elements.
    H_hlist = np.zeros([4,4])
    for i in range(N+n):
        # H_hlist = Sum_j |Uhj>><<Uhj| -- 1/4 can be ignored
        Uhi = Uh_list[i]
        H_hlist = H_hlist + np.kron(Uhi, np.eye(2)) @ bell_state @ bell_state.conj().T @ np.kron(Uhi, np.eye(2)).conj().T
    # Calculate  <<Uhi | H_hlist | Uhi>> for each Uhi
    fh_list = []
    for i in range(N+n):
        Uhi = Uh_list[i]
        fhi = bell_state.conj().T @ np.kron(Uhi, np.eye(2)).conj().T @ H_hlist @ np.kron(Uhi, np.eye(2)) @ bell_state
        fhi = np.real(fhi)
        fh_list.append(fhi)
    # Calculate the smallest n terms -- using function: find_max(given_list)
    Uh_list_final = Uh_list.copy()
    for i in range(n):
        loc_min = find_min(fh_list)
        del fh_list[loc_min]
        del Uh_list_final[loc_min]
    
    return Uh_list_final

def find_min(given_list):
    # Input: A list
    # Output: the location of the smallest number
    location = 0
    num = given_list[0]
    for i in range(len(given_list)):
        if num > given_list[i]:
            num = given_list[i]
            location = i
    
    return location

# 3.3 For dephasing noise, get rid of n noisy outcomes
def random_fromg_geth_dephasing(Ug):
    # Input: Ug: np.matrix, distribution
    # Output: Uh:np.matrix: distribution -- p(h|g) = 1/4 * <<Uh| (|Ug>> <<Ug|/2 + |ZUg>> <<UgZ|/2 ) |Uh>> 
    pgh_max = 1  # p(h|g) always smaller than 1
    state_g_final1 = np.kron(Ug, np.eye(2)) @ bell_state @ (np.kron(Ug, np.eye(2)) @ bell_state).conj().T
    state_g_final2 = np.kron(pauli_z, np.eye(2)) @ np.kron(Ug, np.eye(2)) @ bell_state @ (np.kron(pauli_z, np.eye(2)) @ np.kron(Ug, np.eye(2)) @ bell_state).conj().T
    state_g_final = state_g_final1 + state_g_final2
    while 1:
        Uh = U2_random_Euler_mod()
        pgh = random.uniform(0, pgh_max)
        fgh = 1/4 * (np.kron(Uh, np.eye(2)) @ bell_state).conj().T @ state_g_final @ (np.kron(Uh, np.eye(2)) @ bell_state)  # p(h|g) = 1/4 * <<Uh| (|Ug>> <<Ug|/2 + |ZUg>> <<UgZ|/2 ) |Uh>> 
        fgh = np.real(fgh)
        if pgh < fgh:
            break
    return Uh


- From N measure outcomes {Uh1, ... UhN} get the maximum likelihood Uh -- in Euler angle representation
- Optimize the 3 Euler angles
- Loss function: L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh(alphah, betah, gammah) | Uhi>> |^2
- Two main functions:
    1. For erasure noise, the probability get Uhi: p(hi|g) = 1/4 * | <<Ug|Uhi>> |^2
        - From_g_get_hath(Ug, N)
    2. For non-erasure noise, first need to use majority vote as given above to get rid of the noisy Uhi
        - From_g_get_hath_noise(Ug, N, n, noise)

In [ ]:
# From N measure outcomes {Uh1, ... UhN} get the maximum likelihood Uh -- in Euler angle representation
# Optimization is done via "Paddle Quantum"

# paddle quantun ancilla functions
# Generate the parameterized quantum network
def circuit_fromhNtoh(N_cir, DEPTH_cir, alpha, beta, gamma):
    # The parameterized quantum network has N_cir=2 qubits, and Depth:EPTH_cir = 3
    cir = UAnsatz(N_cir)
    # The first qubit goes through Uh = Rz(gamma) Ry(beta) Rz(alpha)
    cir.rz(alpha[0], 0)
    cir.ry(beta[0], 0)
    cir.rz(gamma[0], 0)

    return cir

# Forward part: calculate the loss function: L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh|Uhi>> |^2
class Opt_fromhNtoh(paddle.nn.Layer):
    # 1. def __init__(…)；
    def __init__(self, N_cir, DEPTH_cir, H_cir, ini_cir, dtype='float64'):
        super(Opt_fromhNtoh, self).__init__()
        # Initial variables
        self.N_cir = N_cir
        self.DEPTH_cir = DEPTH_cir
        self.alpha = self.create_parameter(shape=[1],  # 角度参数alpha
                default_initializer=paddle.nn.initializer.Uniform(low=0., high=2*np.pi), 
                dtype=dtype, is_bias=False)
        self.beta = self.create_parameter(shape=[1],  # 角度参数beta
                default_initializer=paddle.nn.initializer.Uniform(low=0., high=np.pi), 
                dtype=dtype, is_bias=False)
        self.gamma = self.create_parameter(shape=[1],  # 角度参数gamma
                default_initializer=paddle.nn.initializer.Uniform(low=0., high=2*np.pi), 
                dtype=dtype, is_bias=False)
        self.H = paddle.to_tensor(H_cir, dtype='complex128')
        self.ini_cir = paddle.to_tensor(ini_cir, dtype='complex128')

    def forward(self):
        # The PQC
        cir = circuit_fromhNtoh(self.N_cir, self.DEPTH_cir, self.alpha, self.beta, self.gamma)
        # The output state
        state_final = cir.run_density_matrix(self.ini_cir)
        # Calculate the loss function
        loss = trace(matmul(self.H, state_final))  # L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh|Uhi>> |^2
        loss = paddle.real(loss)

        return loss, cir



# Main function 1: For erasure noise, the probability get Uhi: p(hi|g) = 1/4 * | <<Ug|Uhi>> |^2
def From_g_get_hath(Ug, N):
    # Given Ug, get the output Uh, N is the RF states number
    # 1. From Ug, get Uhi list: N elements
    # 2. Use paddle quantum to get final Uh
    # Input：Ug (np.matrix)， N：N RF states
    # Output：Uh (np.matrix)
    
    # 0. Parameters in optimization
    ITR = 125       # The number for iteration
    LR = 0.3        # Learning rate
    SEED = 100        # random seed
    paddle.seed(SEED)  # seed for paddle quantum
    
    # 1. From Ug, get Uhi list: N elements
    Uhi_list = measoutcome_gN(Ug, N)
    
    # 2. Use paddle quantum to get final Uh
    # 2.1 For loss function: L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh|Uhi>> |^2
        # L = Tr[rho_out \cdot H_cir]
        # H_cir：Sum_i=1^N (-) 1/2 |Uhi>><<Uhi| -- in optimization, we do minimization
    H_cir = np.zeros([4,4])
    for i in range(N):
        # Get Sum_i=1^N - 1/2 |Uhi>><<Uhi| -- 1/2 is in bell_state
        Uhi = Uhi_list[i]
        H_cir = H_cir - np.kron(Uhi, np.eye(2)) @ bell_state @ bell_state.conj().T @ np.kron(Uhi, np.eye(2)).conj().T
    # 2.2 Forward network
    ini_cir = bell_state @ bell_state.conj().T
    N_cir = 2
    DEPTH_cir = 3
    
    # 2.3 Backward optimization
    loss_list = []
    alphah_list = []
    betah_list = []
    gammah_list = []
    myLayer = Opt_fromhNtoh(N_cir, DEPTH_cir, H_cir, ini_cir)
    # The optimizer is chosen to be Adam
    opt = paddle.optimizer.Adam(learning_rate = LR, parameters = myLayer.parameters())    
    # Itertation
    for itr in range(ITR):
        # Calculate the loss function in each iteration
        loss = myLayer()[0]
        # Modify the parameters
        loss.backward()
        opt.minimize(loss)
        opt.clear_grad()
        # recode the learning curves
        loss_list.append(loss.numpy()[0])
        alphah_list.append(myLayer.parameters()[0][0].numpy())
        betah_list.append(myLayer.parameters()[1][0].numpy())
        gammah_list.append(myLayer.parameters()[2][0].numpy())
    
    # Get the final Uh
    Uh = U2_Euler(alphah_list[-1][0], betah_list[-1][0], gammah_list[-1][0])
    
    return Uh



# Main function 2: For non-erasure noise, first need to use majority vote as given above to get rid of the noisy Uhi
def From_g_get_hath_noise(Ug, N, n, noise):
    # Given Ug, get the output Uh, 
    # N is the RF states number, n: noisy RF states number, noisy: noisy models
        # noise = 1: completely depolarizing
        # noise = 2:completely dephasing
    # 1. From Ug, get Uhi list: N elements
    # 2. Use paddle quantum to get final Uh
    # Output：Uh (np.matrix)

    
    # 0. Parameters in optimization
    ITR = 125       # The number for iteration
    LR = 0.3        # Learning rate
    SEED = 100        # random seed
    paddle.seed(SEED)  # seed for paddle quantum
    
    # 1. From Ug, get Uhi list: N elements
    Uhi_list_withnoise = measoutcome_gN(Ug, N)
    # For completely depolarizing noise
    if noise == 1:
        # Add n random Uhi to the list
        for i_noise in range(n):
            Uhi_list_withnoise.append(U2_random_Euler_mod())
    # For completely dephasing noise
    if noise == 2:
        # Add n Uhi to the list in distribution p(h|g) = 1/4 * <<Uh| (|Ug>> <<Ug|/2 + |ZUg>> <<UgZ|/2 ) |Uh>> 
        for i_noise in range(n):
            Uhi_list_withnoise.append(random_fromg_geth_dephasing(Ug))
    # Get rid of the n noisy outcomes
    Uhi_list = getrid_dep(Uhi_list_withnoise, N, n)
    
    # 2. Use paddle quantum to get final Uh
    # 2.1 For loss function: L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh|Uhi>> |^2
        # L = Tr[rho_out \cdot H_cir]
        # H_cir：Sum_i=1^N (-) 1/2 |Uhi>><<Uhi| -- in optimization, we do minimization
    H_cir = np.zeros([4,4])
    for i in range(N):
        # Get Sum_i=1^N - 1/2 |Uhi>><<Uhi| -- 1/2 is in bell_state
        Uhi = Uhi_list[i]
        H_cir = H_cir - np.kron(Uhi, np.eye(2)) @ bell_state @ bell_state.conj().T @ np.kron(Uhi, np.eye(2)).conj().T
    # 2.2 Forward network
    ini_cir = bell_state @ bell_state.conj().T
    N_cir = 2
    DEPTH_cir = 3
    
    # 2.3 Backward optimization
    loss_list = []
    alphah_list = []
    betah_list = []
    gammah_list = []
    myLayer = Opt_fromhNtoh(N_cir, DEPTH_cir, H_cir, ini_cir)
    # The optimizer is chosen to be Adam
    opt = paddle.optimizer.Adam(learning_rate = LR, parameters = myLayer.parameters())    

    # Itertation
    for itr in range(ITR):
        # Calculate the loss function in each iteration
        loss = myLayer()[0]
        # Modify the parameters
        loss.backward()
        opt.minimize(loss)
        opt.clear_grad()
        # recode the learning curves
        loss_list.append(loss.numpy()[0])
        alphah_list.append(myLayer.parameters()[0][0].numpy())
        betah_list.append(myLayer.parameters()[1][0].numpy())
        gammah_list.append(myLayer.parameters()[2][0].numpy())
    # Get the final Uh
    Uh = U2_Euler(alphah_list[-1][0], betah_list[-1][0], gammah_list[-1][0])
    
    return Uh


### 3' $|U_g / \sqrt{2} >> ^{\otimes N} $ pass through IID p-depolarizing, get final measurement outcome $U_h$
- This part deals with RF states pass through IID p-depolarizing noise
- From Ug give final Uh
- From N measure outcomes {Uh1, ... UhN} get the maximum likelihood Uh -- in Euler angle representation

In [ ]:
# IID noisy model (strong noise model), with probability (1-p) RF  doesn't change, with probability p becomes max mixed

def random_fromg_geth_strong(Ug, p):
    # Input: Ug: np.matrix, distribution
    # Output: Uh:np.matrix: distribution -- p(h|g) = (1-p) * 1/4 * | <<Ug|Uh>> |^2  +  p / 4
    pgh_max = 1  # p(h|g) always smaller than 1
    while 1:
        Uh = U2_random_Euler_mod()
        pgh = random.uniform(0, pgh_max)
        fgh = (1-p) * 1/4 * ((Ug.conj().T @ Uh).trace()) * ((Ug.conj().T @ Uh).trace().conj()) + p/4  # (1-p) * 1/4 * | <<Ug|Uh>> |^2  +  p / 4
        fgh = np.real(fgh)
        if pgh < fgh:
            break
    return Uh

def measoutcome_gN_strong(Ug, N, p):
    # Input: Ug: np.matrix, distribution
    # Output: a list {Uh1, ... UhN}:np.matrix: distribution -- p(h|g) = (1-p) * 1/4 * | <<Ug|Uh>> |^2  +  p / 4
    Uh_list = []
    for i in range(N):
        Uh = random_fromg_geth_strong(Ug, p)
        Uh_list.append(Uh)
    
    return Uh_list



- From N measure outcomes {Uh1, ... UhN} get the maximum likelihood Uh -- in Euler angle representation

In [ ]:
def From_g_get_hath_noise_strong(Ug, N, p):
    # IID noisy model (strong noise model), with probability (1-p) RF  doesn't change, with probability p becomes max mixed
    # Given Ug, get the output Uh, N is the RF states number
    # 1. From Ug, get Uhi list: N elements
    # 2. Use paddle quantum to get final Uh
    # Input：Ug (np.matrix)， N：N RF states, p
    # Output：Uh (np.matrix)
    
    # 0. Parameters in optimization
    ITR = 155       # The number for iteration
    LR = 0.3        # Learning rate
    SEED = 100        # random seed
    paddle.seed(SEED)  # seed for paddle quantum
    
    # 1. From Ug, get Uhi list: N elements
    Uhi_list_withnoise = measoutcome_gN_strong(Ug, N, p)
    
    # 2. Use paddle quantum to get final Uh
    # 2.1 For loss function: L(alphah, betah, gammah) = Sum_i=1^N 1/4 * | <<Uh|Uhi>> |^2
        # L = Tr[rho_out \cdot H_cir]
        # H_cir：Sum_i=1^N (-) 1/2 |Uhi>><<Uhi| -- in optimization, we do minimization
    H_cir = np.zeros([4,4])
    for i in range(N):
        # Get Sum_i=1^N - 1/2 |Uhi>><<Uhi| -- 1/2 is in bell_state
        Uhi = Uhi_list_withnoise[i]
        H_cir = H_cir - np.kron(Uhi, np.eye(2)) @ bell_state @ bell_state.conj().T @ np.kron(Uhi, np.eye(2)).conj().T
    # 2.2 Forward network
    ini_cir = bell_state @ bell_state.conj().T
    N_cir = 2
    DEPTH_cir = 3
    
    # 2.3 Backward optimization
    loss_list = []
    alphah_list = []
    betah_list = []
    gammah_list = []
    myLayer = Opt_fromhNtoh(N_cir, DEPTH_cir, H_cir, ini_cir)
    # The optimizer is chosen to be Adam
    opt = paddle.optimizer.Adam(learning_rate = LR, parameters = myLayer.parameters())    

    # Itertation
    for itr in range(ITR):
        # Calculate the loss function in each iteration
        loss = myLayer()[0]
        # Modify the parameters
        loss.backward()
        opt.minimize(loss)
        opt.clear_grad()
        # recode the learning curves
        loss_list.append(loss.numpy()[0])
        alphah_list.append(myLayer.parameters()[0][0].numpy())
        betah_list.append(myLayer.parameters()[1][0].numpy())
        gammah_list.append(myLayer.parameters()[2][0].numpy())
    # Get the final Uh
    Uh = U2_Euler(alphah_list[-1][0], betah_list[-1][0], gammah_list[-1][0])
    
    return Uh

### 4. About coding -- functions related to 5-qubit code

In [ ]:
# Some related functions for encoding, decoding

# Binary to Decimal ---- for encoding
def TwoToTen(a_two):
    # Input: Binary string:  a_two
    # Output: Decimal number: a_ten
    a_ten = 0
    for i in range(len(a_two)):
        new = int(a_two[len(a_two)-i-1])  # Get the last "i+1"-th number
        a_ten = a_ten + new * (2**i)
    
    return a_ten

# Decimal to Binary ---- for decoding
def TenToTwo(a_ten, n):
    # Input: Decimal number: a_ten, n: the length of the output string
    # Output: Binary string:  a_two
    a_two = ''
    while a_ten:
        new = a_ten % 2
        new = str(new)
        a_two += new
        a_ten = a_ten // 2
    if len(a_two) < n:
        for i in range(n - len(a_two)):
            a_two += '0'
    a_two = a_two[::-1]
    
    return a_two

# Get the measure probability and output state for the 5-qubit code
# state is in order: 4 measure qubits \otimes 5 code qubits \otimes 1 ancilla qubit
def MeasFiveQ(final_state, basis):
    # Input: final_state: 10-qubit code state after noise (numpy.matrix), basis: wanted measure outcome 
        # 0 for 0000, 1 for 0001,... 15 for 1111
    # Output: The measure probability and the remaining 6-qubit state: 5 code qubits \otimes 1 ancilla qubit (numpy.matrix)
        # pr, meas_prob_basis
    meas_prob_basis = np.kron(vec(basis, 4), np.eye(2**6)) @ final_state @ np.kron(vec(basis, 4).conj().T, np.eye(2**6))
    pr = np.real(meas_prob_basis.trace())
    if pr > 0:  # pr should not be 0
        meas_prob_basis = meas_prob_basis / pr  # 归一化
    
    return pr, meas_prob_basis

# Circuit for Stabilizer measure for 5-qubit code (in paddle quantum form)
def stabilizer_meas(cir):
    # Input: 10-qubit initial circuit
    # Four initial H gates on measure qubits
    for i in range(4):
        cir.h(i)
    # 1：XZZXI
    cir.cnot([0,4])
    cir.cz([0,5])
    cir.cz([0,6])
    cir.cnot([0,7])
    # 2：IXZZX
    cir.cnot([1,5])
    cir.cz([1,6])
    cir.cz([1,7])
    cir.cnot([1,8])
    # 3：XIXZZ
    cir.cnot([2,6])
    cir.cz([2,7])
    cir.cz([2,8])
    cir.cnot([2,4])
    # 4：ZXIXZ
    cir.cnot([3,7])
    cir.cz([3,8])
    cir.cz([3,4])
    cir.cnot([3,5])
    # Four final H gates on measure qubits
    for i in range(4):
        cir.h(i)
        
    return None

# Do correction in decoding according to measure outcomes
def correct_accoring_tomeas(meas_out, outstate):
    # Input: meas_out: measure outcome (string)
            # outstate (np.matrix), 5 code \otimes 1 anc
    # Output: final_state: state after correction (np.matrix)

    if meas_out == '0000':
        U_cor = np.eye(2**6)
    elif meas_out == '0001':
        U_cor = np.kron(pauli_x, np.eye(2**5))
    elif meas_out == '1000':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_x, np.eye(2**4)))
    elif meas_out == '1100':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_x, np.eye(2**3)))    
    elif meas_out == '0110':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_x, np.eye(2**2)))
    elif meas_out == '0011':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_x, np.eye(2**1)))
    elif meas_out == '1010':
        U_cor = np.kron(pauli_z, np.eye(2**5))
    elif meas_out == '0101':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_z, np.eye(2**4)))
    elif meas_out == '0010':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_z, np.eye(2**3)))
    elif meas_out == '1001':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_z, np.eye(2**2)))
    elif meas_out == '0100':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_z, np.eye(2**1)))
    elif meas_out == '1011':
        U_cor = np.kron(pauli_y, np.eye(2**5))
    elif meas_out == '1101':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_y, np.eye(2**4)))
    elif meas_out == '1110':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_y, np.eye(2**3)))
    elif meas_out == '1111':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_y, np.eye(2**2)))
    else:  # meas_out == '0111':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_y, np.eye(2**1)))
        
    # The final state
    final_state = U_cor @ outstate @ U_cor.conj().T
    
    return final_state


### 5. About coding -- get the state after Ug encoding
- 5-qubit code encoding algorithm
- The order of the space: 4 meas * 5 code * 1 anc

In [ ]:
# Some vectors and matrices used in Encoding
# 5-qubit Encoding
# |0_L> = 1/4*(00000 + 10010 + 01001 + 10100 + 01010 - 11011 - 00110 - 11000 - 11101 - 00011 - 11110 - 01111 - 10001 - 01100 - 10111 + 00101)
# |1_L> = 1/4*(11111 + 01101 + 10110 + 01011 + 10101 - 00100 - 11001 - 00111 - 00010 - 11100 - 00001 - 10000 - 01110 - 10011 - 01000 + 11010)
    # Note: the expression from wiki has some problems
    
# Generate |0_L>, |1_L>: code_0, code_1
# |0_L>，(np.ket)
code_0 = np.zeros([2**5, 1])
code_0_list_plus = ['00000', '10010', '01001', '10100', '01010', '00101']
code_0_list_minus = ['11011', '00110', '11000', '11101', '00011', '11110', '01111', '10001', '01100', '10111']
for i in range(len(code_0_list_plus)):
    code_0[TwoToTen(code_0_list_plus[i])] = 1/4
for i in range(len(code_0_list_minus)):
    code_0[TwoToTen(code_0_list_minus[i])] = -1/4
# |0_L><0_L| density matrix
code_0_density = code_0 @ code_0.conj().T

# |1_L>，(np.ket)
code_1 = np.zeros([2**5, 1])
code_1_list_plus = ['11111', '01101', '10110', '01011', '10101', '11010']
code_1_list_minus = ['00100', '11001', '00111', '00010', '11100', '00001', '10000', '01110', '10011', '01000']
for i in range(len(code_1_list_plus)):
    code_1[TwoToTen(code_1_list_plus[i])] = 1/4
for i in range(len(code_1_list_minus)):
    code_1[TwoToTen(code_1_list_minus[i])] = -1/4
# |1_L><1_L| density matrix
code_1_density = code_1 @ code_1.conj().T
    
# The Encoding isometry: V_Eori
# V_E0 = |0_L><0| + |1_L><1|
V_Eori = code_0 @ vec(0,1) + code_1 @ vec(1,1)

In [ ]:
# Get the state after encoding:
# For covarient encoding: V_Ecov = (Ug)^\otimes 5 V_E0 Ug^dag
# Hilbert space order: 4 meas * 5 code * 1 anc

def state_afterencoding(Ug):
    # Input: Ug
    # Output: input_total (paddle.matrix), final state in "4 meas * 5 code * 1 anc"
    
    # (Ug)^\otimes 5
    Ug5 = Ug
    for i in range(4):
        Ug5 = np.kron(Ug5, Ug)  # (Ug)^\otimes 5
    # V_Ecov = (Ug)^\otimes 5 V_E0 Ug^dag
    V_Ecov = Ug5 @ V_Eori @ Ug.conj().T
    
    # Get the state after encoding
    # Initial state: 1 logical \otimes 1 ancilla
    logical_ini = bell_state
    # After encoding: state：input_code, input_total
    input_code = np.kron(V_Ecov, np.eye(2)) @ logical_ini  # code \otimes anc，(np.ket)
    input_code_density = input_code @ input_code.conj().T  # code \otimes anc，(np.matrix)
    input_total = np.kron(vec(0, 4).conj().T, input_code)  # add 4 measure qubits
    input_total = input_total * input_total.conj().T  # (np.matrix)
    input_total = paddle.to_tensor(input_total, dtype = 'complex128')  # change to paddle density
    
    return input_total


### 6. About coding -- after noise and Uh decoding, calculate the entanglement fidelity

In [ ]:
def fidelity_hg(Ug, Uh, input_total, noise):
    # Input: Ug, Uh -- np.matrix，
            # input_total: state after encoding -- paddle.matrix
            # noise model
                # noise = 1，a code qubit pass through the completely depolarizing noise
                # noise = 2，a code qubit pass through the completely dephasing noise
    # Output: fidelityh: Entanglement error with Ug encoding and Uh decoding
    
    # 1. Get the state after noise -- state_afternoise (np.matrix)
    # The input is input_total (paddle.matrix):  4 mea \otimes 5 code \otimes 1 anc
    # Go through the noise channel
    cir_noise = UAnsatz(10)  # This code has 10 qubit -- 4 measure, 5 code, 1 ancilla
    if noise == 1:
        # a code qubit pass through the completely depolarizing noise, p = 3/4
        cir_noise.depolarizing(3/4, 5)
    if noise == 2:
        # a code qubit pass through the completely dephasing noise, p = 3/4
        cir_noise.phase_flip(1/2, 5)
    state_afternoise = cir_noise.run_density_matrix(input_total).numpy()  # numpy.matrix

    
    # 2. Pass through decoding: Dec_h = Uh @ Dec @ Uh5^dag
    # 2.1 Go through Uh5^dag
    Uh5 = Uh
    for i in range(4):
        Uh5 = np.kron(Uh5, Uh)  # get (Uh)^\otimes 5
    # Get state_afterUhdag: 4 mea \otimes 5 code \otimes 1 anc
    # Apply eye(16) \otimes Uh5 \otimes eye(2))
    state_afterUhdag = np.kron(np.kron(np.eye(16), Uh5.conj().T), np.eye(2)) @ state_afternoise @ np.kron(np.kron(np.eye(16), Uh5.conj().T), np.eye(2)).conj().T
    state_afterUhdag = paddle.to_tensor(state_afterUhdag, dtype = 'complex128')  # change to (paddle.matrix)
    # 2.2 Go through decoding: 
    cir = UAnsatz(10)
    stabilizer_meas(cir)
    final_total_density = cir.run_density_matrix(state_afterUhdag).numpy()  # (numpy.matrix)
    # 2.3 Correct after measure
    # There are 16 measure outcomes, for each we do different corrections
    # Get the probability and output state for each measure outcomes, store in lists
        # prob_meas：32 elements: 16 outcomes (string), and 16 probabilities
        # prob_output：16 elements: 16 output states (numpy.matrix)
    prob_meas = []
    prob_output = []
    for i in range(2**4):
        basis = TenToTwo(i, n=4)
        pr, meas_prob_basis = MeasFiveQ(final_total_density, i)  # measure the first 4 qubits
        prob_meas.append(basis+':')
        prob_meas.append(pr)
        prob_output.append(meas_prob_basis)
        
    # 3. Do correction for each measure outcome and calculate fidelity for each measurement outcome
    fidelityh = 0
    for i in range(2**4):
        # Get state_aftercori, state_afterVdagi, state_afterUhi
        state_aftercori = correct_accoring_tomeas(TenToTwo(i, n=4), prob_output[i])
        state_afterVdagi = np.kron(V_Eori.conj().T, np.eye(2)) @ state_aftercori @ np.kron(V_Eori, np.eye(2))
        state_afterUhi = np.kron(Uh, np.eye(2)) @ state_afterVdagi @ np.kron(Uh.conj().T, np.eye(2))
        # calculate fidelity for each measurement outcome hi
        fidelityhi = np.real(bell_state.conj().T @ state_afterUhi @ bell_state)
        fidelityh += fidelityhi * np.real(prob_meas[2*i + 1])
    fidelityh = fidelityh[0][0]
    
    return fidelityh

    
    

## Numerical experiment part
- iid p-depolarizing noise (every RF state pass through p-depolarizing noise)
- erasure noise
- completely depolarizing noise (one fifth RF states pass through noise)
- completely dephasing noise (one fifth RF states pass through noise)

### 1. iid p-depolarizing noise
- N_total $\in$ [40, 70, 100, 150, 200, 300, 400, 500]
- p=0.2
- Calculate the average entanglement fidelity, with ITR_i_hg = 400 random Ug, store in list f_N_ave_list
- Get the worst-case error by : worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)
    1. Fittint
    2. Get Plot

In [ ]:
# For IID noise: each RF state will has probability p go through a depolarizing, the code space only has one completely depolarizing
# Change: For RF space: From_g_get_hath_noise_strong(Ug, N, p)

# Get the relation between entanglement fidelity and N_total
# ITR_i_hg = 400: Calculate the average entanglement fidelity, with 400 random Ug
    # 1. Randomly generate Ug
    # 2. Get Uh
    # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
    
p = 0.2
ITR_i_hg = 400

# Finally we will get 3 lists: N_total_list, f_N_ave_list, f_N_var_list
N_total_list=[40, 70, 100, 150, 200, 300, 400, 500]  # [40, 70, 100, 150, 200, 300, 400, 500]
f_N_ave_list = []
f_N_var_list = []

# For each N_total in N_total_list, calculate fidelity_ave, fidelity_var
for N_total in N_total_list:
    print('N_total:', N_total)
    time_start = time.time() 

    fidelityh_list =  []  # store fidelityh in each iteration in the list to calculate the average and variance
    for i_hg in range(ITR_i_hg):
        # 1. Randomly generate Ug
        Ug = U2_random_Euler_mod()
        # 2. Get measure outcome Uh
        Uh = From_g_get_hath_noise_strong(Ug, N_total, p)
       # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
        # 3.1 The state after encoding: (Ug)^\otimes 5 V_E0 Ug^dag
        input_total = state_afterencoding(Ug)
        # 3.2 The fidelityh with the above Ug, Uh
        noise = 1
        fidelityh = fidelity_hg(Ug, Uh, input_total, noise)  # depolarizing noise
        fidelityh_list.append(fidelityh)
    # calculate the average and variance for this N_total
    fidelity_ave = np.mean(fidelityh_list)
    fidelity_var = np.var(fidelityh_list)
    print('fidelity_ave:', fidelity_ave)
    print('fidelity_var:', fidelity_var)
    f_N_ave_list.append(fidelity_ave)
    f_N_var_list.append(fidelity_var)
    
    time_span = time.time() - time_start 
    print('time:', time_span)


In [ ]:
# Fit and plot -- for IID noise, p = 0.2

# 1. Use the following list -- change to worst-case error
#N_total_list
err_N_ave_list = [2 * (1-i) for i in f_N_ave_list]  # worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)

# 2. Plot original figure
plt.figure(1)
func1, = plt.plot(N_total_list, err_N_ave_list, 
                  alpha=0.7, marker='', linestyle="-", color='r')
plt.xlabel('n')
plt.ylabel('worst-case error')

plt.legend(handles=[
    func1,
],
    labels=[
        'iid noise (p = 0.2), with n Bell RF state',
    ], loc='best')

plt.show()

# 3. Fit
# The performance is similar as: err_N_ave \app (coe / N)
def func(x, coe):
    return coe / x
Fitx = N_total_list
Fitx = np.array(Fitx)
Fity = err_N_ave_list
Fity = np.array(Fity)
 
# Least squares method
popt, pcov = curve_fit(func, Fitx, Fity)
# Get the fitting coefficient
coe = popt[0] 
yvals = func(Fitx, coe)
print('coe:', coe)
print('pcov:', pcov)
# Plot
Fitxnew = []
Fitynew = []
for i in range(N_total_list[0], N_total_list[-1]+1):
    Fitxnew.append(i)
    Fitynew.append(func(i, coe))
plot1 = plt.plot(Fitx, Fity, 's',label='original values')
plot2 = plt.plot(Fitxnew, Fitynew, 'r',label='polyfit values')
plt.xlabel('n')
plt.ylabel('worst-case error')
plt.legend(handles=[
    func1,
],
    labels=[
        'iid noise (p = 0.2), with n Bell RF state',
    ], loc='best')
plt.show()



### 2. erasure noise
- N $\in$ [40, 70, 100, 150, 200, 300, 400, 500]
- Calculate the average entanglement fidelity, with ITR_i_hg = 400 random Ug, store in list f_N_ave_list
- Get the worst-case error by : worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)
    1. Fittint
    2. Get Plot

In [ ]:
# For erasure noise

# Get the relation between entanglement fidelity and N
# ITR_i_hg = 400: Calculate the average entanglement fidelity, with 400 random Ug
    # 1. Randomly generate Ug
    # 2. Get Uh
    # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh

noise = 1  # Use completely depolarizing noise (same as erasure noise)
ITR_i_hg = 400
    
# Finally we will get 3 lists: N_list, f_N_ave_list, f_N_var_list
N_list=[40, 70, 100, 150, 200, 300, 400, 500]  # [40, 70, 100, 150, 200, 300, 400, 500]
f_N_ave_list = []
f_N_var_list = []

# For each N in N_list, calculate fidelity_ave, fidelity_var
for N in N_list:
    print('N:', N)
    time_start = time.time() 

    fidelityh_list =  []  # store fidelityh in each iteration in the list to calculate the average and variance
    for i_hg in range(ITR_i_hg):
        # 1. Randomly generate Ug
        Ug = U2_random_Euler_mod()
        # 2. Get measure outcome Uh
        Uh = From_g_get_hath(Ug, N)
        # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
        # 3.1 The state after encoding: (Ug)^\otimes 5 V_E0 Ug^dag
        input_total = state_afterencoding(Ug)
        # 3.2 The fidelityh with the above Ug, Uh
        fidelityh = fidelity_hg(Ug, Uh, input_total, noise)  # depolarizing noise
        fidelityh_list.append(fidelityh)
    # calculate the average and variance for this N
    fidelity_ave = np.mean(fidelityh_list)
    fidelity_var = np.var(fidelityh_list)
    print('fidelity_ave:', fidelity_ave)
    print('fidelity_var:', fidelity_var)
    f_N_ave_list.append(fidelity_ave)
    f_N_var_list.append(fidelity_var)
    
    time_span = time.time() - time_start 
    print('time:', time_span)
    



In [ ]:
# Fit and plot -- for erasure

# 1. Use the following list -- change to worst-case error
#N_list
err_N_ave_list = [2 * (1-i) for i in f_N_ave_list]  # worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)

# 2. Plot original figure
plt.figure(1)
func1, = plt.plot(N_list, err_N_ave_list, 
                  alpha=0.7, marker='', linestyle="-", color='r')
plt.xlabel('n')
plt.ylabel('worst-case error')

plt.legend(handles=[
    func1,
],
    labels=[
        'erasure noise, with Bell RF state',
    ], loc='best')

plt.show()

# 3. Fit
# The performance is similar as: err_N_ave \app (coe / N)
def func(x, coe):
    return coe / x
Fitx = N_list
Fitx = np.array(Fitx)
Fity = err_N_ave_list
Fity = np.array(Fity)
 
# Least squares method
popt, pcov = curve_fit(func, Fitx, Fity)
# Get the fitting coefficient
coe = popt[0] 
yvals = func(Fitx, coe)
print('coe:', coe)
print('pcov:', pcov)
# Plot
Fitxnew = []
Fitynew = []
for i in range(N_list[0], N_list[-1]+1):
    Fitxnew.append(i)
    Fitynew.append(func(i, coe))
plot1 = plt.plot(Fitx, Fity, 's',label='original values')
plot2 = plt.plot(Fitxnew, Fitynew, 'r',label='polyfit values')
plt.xlabel('n')
plt.ylabel('worst-case error')
plt.legend(handles=[
    func1,
],
    labels=[
        'erasure noise, with Bell RF state',
    ], loc='best')
plt.show()


### 3. completely depolarizing noise
- N_total $\in$ [40, 70, 100, 150, 200, 300, 400, 500]
- one fifth RF states pass through noise
- Calculate the average entanglement fidelity, with ITR_i_hg = 400 random Ug, store in list f_N_ave_list
- Get the worst-case error by : worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)
    1. Fittint
    2. Get Plot

In [ ]:
# For depolarizing noise, n = 0.2 Ntotal

# Get the relation between entanglement fidelity and N_total
# ITR_i_hg = 400: Calculate the average entanglement fidelity, with 400 random Ug
    # 1. Randomly generate Ug
    # 2. Get Uh
    # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
    
noise = 1  # completely depolarizing noise
ITR_i_hg = 400

# Finally we will get 3 lists: N_total_list, f_N_ave_list, f_N_var_list
N_total_list=[40, 70, 100, 150, 200, 300, 400, 500]  # [40, 70, 100, 150, 200, 300, 400, 500]
f_N_ave_list = []
f_N_var_list = []

# For each N_total in N_total_list, calculate fidelity_ave, fidelity_var
for N_total in N_total_list:
    print('N_total:', N_total)
    n = N_total // 5  # n = 0.2 Ntotal
    N = N_total - n
    time_start = time.time() 

    fidelityh_list =  []  # store fidelityh in each iteration in the list to calculate the average and variance
    for i_hg in range(ITR_i_hg):
        # 1. Randomly generate Ug
        Ug = U2_random_Euler_mod()
        # 2. Get measure outcome Uh
        Uh = From_g_get_hath_noise(Ug, N, n, noise)
        # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
        # 3.1 The state after encoding: (Ug)^\otimes 5 V_E0 Ug^dag
        input_total = state_afterencoding(Ug)
        # 3.2 The fidelityh with the above Ug, Uh
        fidelityh = fidelity_hg(Ug, Uh, input_total, noise)  # depolarizing noise
        fidelityh_list.append(fidelityh)
    # calculate the average and variance for this N_total
    fidelity_ave = np.mean(fidelityh_list)
    fidelity_var = np.var(fidelityh_list)
    print('fidelity_ave:', fidelity_ave)
    print('fidelity_var:', fidelity_var)
    f_N_ave_list.append(fidelity_ave)
    f_N_var_list.append(fidelity_var)
    
    time_span = time.time() - time_start 
    print('time:', time_span)



In [ ]:
# Fit and plot -- for depolarizing noise: one fifth RF go through noise

# 1. Use the following list -- change to worst-case error
#N_total_list
err_N_ave_list = [2 * (1-i) for i in f_N_ave_list]  # worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)

# 2. Plot original figure
plt.figure(1)
func1, = plt.plot(N_total_list, err_N_ave_list, 
                  alpha=0.7, marker='', linestyle="-", color='r')
plt.xlabel('n')
plt.ylabel('worst-case error')

plt.legend(handles=[
    func1,
],
    labels=[
        'depolarizing noise, with Bell RF state',
    ], loc='best')

plt.show()

# 3. Fit
# The performance is similar as: err_N_ave \app (coe / N)
def func(x, coe):
    return coe / x
Fitx = N_total_list
Fitx = np.array(Fitx)
Fity = err_N_ave_list
Fity = np.array(Fity)
 
# Least squares method
popt, pcov = curve_fit(func, Fitx, Fity)
# Get the fitting coefficient
coe = popt[0] 
yvals = func(Fitx, coe)
print('coe:', coe)
print('pcov:', pcov)
# Plot
Fitxnew = []
Fitynew = []
for i in range(N_total_list[0], N_total_list[-1]+1):
    Fitxnew.append(i)
    Fitynew.append(func(i, coe))
plot1 = plt.plot(Fitx, Fity, 's',label='original values')
plot2 = plt.plot(Fitxnew, Fitynew, 'r',label='polyfit values')
plt.xlabel('n')
plt.ylabel('worst-case error')
plt.legend(handles=[
    func1,
],
    labels=[
        'depolarizing noise, with Bell RF state',
    ], loc='best')
plt.show()



### 4. completely dephasing noise
- N_total $\in$ [40, 70, 100, 150, 200, 300, 400, 500]
- one fifth RF states pass through noise
- Calculate the average entanglement fidelity, with ITR_i_hg = 400 random Ug, store in list f_N_ave_list
- Get the worst-case error by : worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)
    1. Fittint
    2. Get Plot

In [ ]:
# For dephasing noise, n = 0.2 Ntotal
    # noise = 2

# Get the relation between entanglement fidelity and N_total
# ITR_i_hg = 400: Calculate the average entanglement fidelity, with 400 random Ug
    # 1. Randomly generate Ug
    # 2. Get Uh
    # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
    
noise = 2  # completely dephasing noise
ITR_i_hg = 400

# Finally we will get 3 lists: N_total_list, f_N_ave_list, f_N_var_list
N_total_list=[40, 70, 100, 150, 200, 300, 400, 500]  # [40, 70, 100, 150, 200, 300, 400, 500]
f_N_ave_list = []
f_N_var_list = []

# For each N_total in N_total_list, calculate fidelity_ave, fidelity_var
for N_total in N_total_list:
    print('N_total:', N_total)
    n = N_total // 5  # n = 0.2 Ntotal
    N = N_total - n
    time_start = time.time() 

    fidelityh_list =  []  # store fidelityh in each iteration in the list to calculate the average and variance
    for i_hg in range(ITR_i_hg):
        # 1. Randomly generate Ug
        Ug = U2_random_Euler_mod()
        # 2. Get measure outcome Uh
        Uh = From_g_get_hath_noise(Ug, N, n, noise)
        # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
        # 3.1 The state after encoding: (Ug)^\otimes 5 V_E0 Ug^dag
        input_total = state_afterencoding(Ug)
        # 3.2 The fidelityh with the above Ug, Uh
        fidelityh = fidelity_hg(Ug, Uh, input_total, noise)  # dephasing noise
        fidelityh_list.append(fidelityh)
    # calculate the average and variance for this N_total
    fidelity_ave = np.mean(fidelityh_list)
    fidelity_var = np.var(fidelityh_list)
    print('fidelity_ave:', fidelity_ave)
    print('fidelity_var:', fidelity_var)
    f_N_ave_list.append(fidelity_ave)
    f_N_var_list.append(fidelity_var)
    
    time_span = time.time() - time_start 
    print('time:', time_span)



In [ ]:
# Fit and plot -- for dephasing noise: one fifth RF go through noise

# 1. Use the following list -- change to worst-case error
#N_total_list
err_N_ave_list = [2 * (1-i) for i in f_N_ave_list]  # worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)

# 2. Plot original figure
plt.figure(1)
func1, = plt.plot(N_total_list, err_N_ave_list, 
                  alpha=0.7, marker='', linestyle="-", color='r')
plt.xlabel('n')
plt.ylabel('worst-case error')

plt.legend(handles=[
    func1,
],
    labels=[
        'dephasing noise, with Bell RF state',
    ], loc='best')

plt.show()

# 3. Fit
# The performance is similar as: err_N_ave \app (coe / N)
def func(x, coe):
    return coe / x
Fitx = N_total_list
Fitx = np.array(Fitx)
Fity = err_N_ave_list
Fity = np.array(Fity)
 
# Least squares method
popt, pcov = curve_fit(func, Fitx, Fity)
# Get the fitting coefficient
coe = popt[0] 
yvals = func(Fitx, coe)
print('coe:', coe)
print('pcov:', pcov)
# Plot
Fitxnew = []
Fitynew = []
for i in range(N_total_list[0], N_total_list[-1]+1):
    Fitxnew.append(i)
    Fitynew.append(func(i, coe))
plot1 = plt.plot(Fitx, Fity, 's',label='original values')
plot2 = plt.plot(Fitxnew, Fitynew, 'r',label='polyfit values')
plt.xlabel('n')
plt.ylabel('worst-case error')
plt.legend(handles=[
    func1,
],
    labels=[
        'dephasing noise, with Bell RF state',
    ], loc='best')
plt.show()
